This can read from a locally-staged hail table directory, or use one in GCS. The table must have a pre-computed `.info.VRS` field. This is computed using `tgg_methods` `vrs_annotation_batch.py`, which uses the `vrs-python` `vcf_annotation.py`

https://github.com/broadinstitute/tgg_methods/blob/master/vrs/vrs_annotation_batch.py (last checked at `a0002f02fbd5dd25487b261e94081a3daec29c64`)

This uses gnomad_methods functions `gnomad_gks` and `add_gks_vrs`, and `add_gks_va`, to return GKS structures for hail tables that are in the gnomad v3 schema, off the branch in this pull request: https://github.com/broadinstitute/gnomad_methods/pull/556

The JSON schema is in `ga4gh/va-spec` on the `gk-pilot` branch

In [1]:
import hail as hl
import json
from typing import List, Tuple
import pandas as pd


Loading BokehJS ...

In [2]:
# The clingen-public bucket will only be the week of 2023-09-18
# The clingen-public-requesterpays bucket will remain available but the client will need to
# be authenticated with a Google Cloud account with billing enabled to pay network transfer fees

# configuration for data outputs
bucket = "clingen-public-requesterpays"
bucket = "clingen-public"

# Writes inputs array as a hail table to this destination, if not None.
# This can be useful for other testing, using this hail table as input without reconstructing it
# inputs_ht_destination_url = f"gs://{bucket}/gnomad-gks-qc/gnomad-filtered-inputs.ht"
inputs_ht_destination_url = None

# Copies output annotations as newline delimited json to this url, if not None
outputs_destination_file = f"gs://{bucket}/gnomad-gks-qc/outputs.ndjson"


# ht_url can be a gs:// path, or a file:// local path

### V3 tables
# Publicly readable, but doesn't have all gnomad variants in it
ht_url = f"gs://{bucket}/gnomad-gks-downsampled-100k.ht"

# Can refer to a local hail table
# ht_url = "../downsample_to_100k_full_release.ht"

# ht_url = "gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht"

ht_url = "gs://clingen-gnomad-mirror/gnomad.genomes.v3.1.2.sites_vrs.ht"
ht_version="3.1.4"

### V4 tables

ht_url = "gs://gcp-public-data--gnomad/release/4.0/ht/exomes/gnomad.exomes.v4.0.sites.ht"
# ht_url = "gs://gcp-public-data--gnomad/release/4.0/ht/genomes/gnomad.genomes.v4.0.sites.ht/"

ht_version = "4.0.0"

print(ht_url)
ht = hl.read_table(ht_url)


Initializing Hail with default parameters...


gs://gcp-public-data--gnomad/release/4.0/ht/exomes/gnomad.exomes.v4.0.sites.ht


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/Users/kferrite/dev/gnomad_methods/venv/lib/python3.11/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://wm12f-58b.broadinstitute.org:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.125-6e6f46797aed
LOGGING: writing to /Users/kferrite/dev/gnomad_methods/notebooks/hail-20231128-1116-0.2.125-6e6f46797aed.l

In [3]:
def loci_alleles_to_pandas(loci: List[hl.Locus], alleles: List[List[str]]) -> pd.DataFrame:
    return pd.DataFrame({
        "locus": loci,
        "alleles": alleles
    })

def filter_table_to_loci_alleles(ht: hl.Table, loci: List[hl.Locus], alleles: List[List[str]]) -> hl.Table:
    left = hl.Table.from_pandas(loci_alleles_to_pandas(loci, alleles))
    left = left.key_by(left.locus, left.alleles)
    return left.join(ht)

def make_loci(inputs: List[Tuple[str,int]], reference_genome="GRCh38") -> List[hl.Locus]:
    return list(
        map(lambda contig, pos: hl.locus(contig, pos, reference_genome=reference_genome),
        map(lambda x: x[0], inputs),
        map(lambda x: x[1], inputs)))

# hl.Table.from_pandas(pd.DataFrame({
#     "locus": [hl.locus("chr1", 629844, reference_genome="GRCh38")],
#     "alleles": [["A", "G"]]
# })).key_by("locus", "alleles").join(ht).show()

# filter_table_to_loci_alleles(
#     ht,
#     make_loci([["chr1", 629844]]),
#     [["A", "G"]]).show()


In [4]:
import gnomad
import gnomad.utils.annotations
import gnomad.resources.grch38.gnomad

# reload (re-running this cell will reload modifications to these modules on disk)
import importlib
importlib.reload(gnomad.utils.annotations)
importlib.reload(gnomad.resources.grch38.gnomad)

from gnomad.utils.annotations import add_gks_va, add_gks_vrs
from gnomad.resources.grch38.gnomad import gnomad_gks


In [5]:
# GnomAD 3.1.2
# GRCh38 expressions

inputs = [
    # {"gnomad": "1-55051215-G-GA"},
    # {"gnomad": "1-629844-A-G"},
    # {"gnomad": "1-633440-A-ATCCC"},
    # {"gnomad": "1-37917308-TTATATA-T"},
    # {"gnomad": "1-4807634-TAAAA-T"},

    # {"gnomad": "X-152954317-T-C"} # gnoamd v2
    {"gnomad": "X-152914462-G-C"},
    # {"gnomad": "X-153688862-T-C"}

    {"gnomad": "1-233194352-G-GT"},
    {"gnomad": "1-95571386-C-A"}
]


In [6]:
# # This cell constructs an indexed hail table from the input alleles
# # so the full gnomad dataset can be rapidly filtered by left joining it onto this.
# # Using a .filter() to check membership in the input set is slow because it needs
# # to do a table scan.

input_gnomad_expressions = hl.literal([x["gnomad"] for x in inputs])
input_terms = [x["gnomad"].split("-") for x in inputs]

df = pd.DataFrame(
    {
        "contig": [str("chr" + i[0]) for i in input_terms],
        "position": [int(i[1]) for i in input_terms],
        "ref": [i[2] for i in input_terms],
        "alt": [i[3] for i in input_terms]
    }
)
input_ht = hl.Table.from_pandas(df)
input_ht = (input_ht
    .annotate(
        locus=hl.locus(input_ht.contig, input_ht.position, reference_genome="GRCh38"),
        alleles=hl.array([input_ht.ref, input_ht.alt]))
    .drop("contig", "position", "ref", "alt")
    .key_by("locus", "alleles"))

input_ht.show()


,
locus,alleles
locus<GRCh38>,array<str>
chr1:95571386,"[""C"",""A""]"
chr1:233194352,"[""G"",""GT""]"
chrX:152914462,"[""G"",""C""]"


In [7]:
ht_filtered = ht

# NOTE: To keep table filtered to only the set of input variants, keep this line uncommented
# ht_filtered = input_ht.join(ht)


ht_filtered = ht_filtered.annotate(
    genomic_coordinates = hl.format("%s-%s-%s-%s",
        ht_filtered.locus.contig[3:], # Remove 'chr'
        hl.str(ht_filtered.locus.position),
        ht_filtered.alleles[0],
        ht_filtered.alleles[1]
    )
)

# # Write the filtered gnomad table to storage
# if inputs_ht_destination_url:
#     ht_filtered.write(inputs_ht_destination_url, overwrite=True)


In [8]:
# Parameters for gnomad_gks/get_gks
ancestry_group_short_names = gnomad.resources.grch38.gnomad.POPS["v3"]
ancestry_groups_full_name_map = gnomad.sample_qc.ancestry.POP_NAMES
gnomad_version_label = "3.1.4"


In [9]:
vrs_variants = []
select_fields = {
    "freq": ht_filtered.freq,
    # "grpMaxFAF95": ht_filtered.grpMaxFAF95,
    "info": ht_filtered.info,
    "genomic_coordinates": ht_filtered.genomic_coordinates,
    "vrs": ht_filtered.info.vrs
}
# if "jointGrpMaxFAF95" in ht_filtered.row:
#     select_fields["jointGrpMaxFAF95"] = ht_filtered.jointGrpMaxFAF95

records = ht_filtered.select(**select_fields)
records = records.take(5)

variant_strs = [r.genomic_coordinates for r in records]
loci = [
    hl.locus(contig=str("chr" + v0), pos=int(v1), reference_genome="GRCh38")
    for [v0, v1, *_] in
    [v.split("-") for v in variant_strs]
]

print(f"gnomAD allele strings: {variant_strs}")


gnomAD allele strings: ['1-11994-T-C', '1-12016-G-A', '1-12060-CTGGAG-C', '1-12074-T-C', '1-12102-G-A']


In [10]:
# This cell takes one locus present in the input table, performs VRS and VA annotation and prints it.

# For performance on larger datasets, the interval should be much larger, at least a few thousand.

ivl_0 =hl.locus_interval(loci[0].contig, loci[0].position, loci[0].position+1, reference_genome="GRCh38")
ht_locus_0 = (hl.filter_intervals(ht_filtered, [ivl_0]))

# ivl_1 =hl.locus_interval(loci[1].contig, loci[1].position, loci[1].position+1, reference_genome="GRCh38")
# ht_locus_1 = (hl.filter_intervals(ht_filtered, [ivl_1]))

ivl_full_chr1 = hl.locus_interval("chr1", 1, 248956422, reference_genome="GRCh38")

import time
t0 = time.time()
gks_annotations = gnomad_gks(
    locus_interval=ivl_0,
    custom_ht=ht_filtered,
    version=ht_version,
    data_type="genomes",
    by_ancestry_group=True,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=False
)
t1 = time.time()
l = len(gks_annotations)
td = t1 - t0
print(f"Annotated {l} records in {td} seconds ({td/l:.6f} sec/rec)")


Annotated 1 records in 6.525036096572876 seconds (6.525036 sec/rec)


/Users/kferrite/dev/gnomad_methods/venv/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning:

Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.



In [11]:
print(json.dumps(gks_annotations[0], indent=2))

for ann in gks_annotations:
    ann_id = ann["gks_va_freq"]["id"]
    with open(f"{ann_id}.json", "w") as f_out:
        json.dump(ann, f_out, indent=4)


{
  "locus": {
    "contig": "chr1",
    "position": 11994,
    "reference_genome": "GRCh38"
  },
  "alleles": [
    "T",
    "C"
  ],
  "gks_vrs_variant": {
    "_id": "ga4gh:VA.nHlWYJXgiuvXrLAxQsraJ6r-daZ8ELL-",
    "type": "Allele",
    "location": {
      "type": "SequenceLocation",
      "sequence_id": "ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO",
      "interval": {
        "start": {
          "type": "Number",
          "value": 11993
        },
        "end": {
          "type": "Number",
          "value": 11994
        },
        "type": "SequenceInterval"
      },
      "_id": "ga4gh:VSL.wuO74hQ9_6ny64lKFgzWuJehwSYZgD-W"
    },
    "state": {
      "type": "LiteralSequenceExpression",
      "sequence": "C"
    }
  },
  "gks_va_freq": {
    "id": "gnomAD-4.0.0-chr1-11994-T-C",
    "type": "CohortAlleleFrequency",
    "label": "Overall Cohort Allele Frequency for chr1-11994-T-C",
    "derivedFrom": {
      "id": "gnomAD4.0.0",
      "type": "DataSet",
      "label": "gnomAD v4

In [12]:
# # This cell does a larger annotation on a chromosome interval (e.g. first 20k bases of APC gene)

# # APC: Chromosome: 5; NC_000005.10 (112707498..112846239)
# apc_start = 112707498
# apc_end = 112846239
# larger_interval = hl.locus_interval(
#     contig="chr5",
#     start=apc_start,
#     end=apc_start + 20000,
#     reference_genome="GRCh38")

# import time
# t0 = time.time()
# gks_annotations = gnomad_gks(
#     locus_interval=larger_interval,
#     custom_ht=ht_filtered,
#     version="3.1.4",
#     data_type="genomes",
#     by_ancestry_group=True,
#     by_sex=True,
#     vrs_only=False,
#     skip_checkpoint=True,
#     skip_coverage=False
# )
# t1 = time.time()
# l = len(gks_annotations)
# td = t1 - t0
# print(f"Annotated {l} records in {td} seconds ({td/l:.6f} sec/rec)")


In [13]:
# print(json.dumps(gks_annotations[2], indent=2))


In [14]:
# # Load the jsonschema that can be used for validating objects
# import subprocess
# import shutil
# import json
# va_spec_clone = "gnomad-gks-v1_va-spec"
# va_spec_branch = "gk-pilot"
# shutil.rmtree(va_spec_clone, ignore_errors=True)
# p = subprocess.run(["git", "clone", "https://github.com/ga4gh/va-spec", "gnomad-gks-v1_va-spec"],
#                    check=True)
# p = subprocess.run(["bash", "-c",
#                     f"cd {va_spec_clone} && git checkout {va_spec_branch}"],
#                    check=True)
# with open(f"{va_spec_clone}/schema/cohortAlleleFreq.json") as f:
#     schema = json.load(f)


In [15]:
import os
import json
import requests
import yaml
import jsonschema

def get_json_http(url):
    r = requests.get(url)
    if r.status_code != 200:
        raise RuntimeError(f"Request failed:\n{r.status_code} {r.content}")
    return json.loads(r.content.decode("utf-8"))

# schema = get_json_http("https://raw.githubusercontent.com/ga4gh/va-spec/gk-pilot/schema/cohortAlleleFreq.json")
# schema = get_json_http("https://raw.githubusercontent.com/ga4gh/va-spec/eea47fd23cd57feac58466702ef3a9a4265ecfa1/schema/cohortAlleleFreq.json")
schema = get_json_http("https://raw.githubusercontent.com/ga4gh/va-spec/1.0-alpha/schema/cohortAlleleFreq.json")

# Local schema
# with open(os.path.expanduser("~/dev/va-spec/schema/cohortAlleleFreq.yaml")) as f:
#     schema = yaml.safe_load(f)


In [17]:
for ann in gks_annotations:
    va_annotation = ann["gks_va_freq"]
    jsonschema.validate(
        instance=va_annotation,
        schema=schema)
